In [1]:
import os
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup

| 測站名稱 | 測站編號 |
| :-------:| :------: |
| **桃園** |  C0C480  |
| **大園** |  C0C540  |
| **觀音** |  C0C590  |
| **平鎮** |  C0C650  |
| **龍潭** |  C0C670  |
| **中壢** |  C0C700  |

In [2]:
# 測站編號，桃園 6 個測站編報
twStationList = ["C0C480", "C0C540", "C0C590", "C0C650", "C0C670", "C0C700"]
# 年份
yearList=['2021']
# 月份
monthSearch = [6, 7, 8, 9, 10, 11]

In [3]:
### 列出年月日
def cdateList(year):
    # days31 (1,3,5,7,8,10,12) days30(2,4,6,9,11)
    month31 = [1,3,5,7,8,10,12]
    yearData = []
    s = ""
    for month in monthSearch:
        if month in month31:
            for day in range(1, 32):
                for hour in range(0, 24):
                    s = year + '/' + str(month).zfill(2) + '/' + str(day).zfill(2) + " " + str(hour).zfill(2) + ":00:00"  ## zfill 指定字串長度
                    yearData.append(s)
        else :
            for day in range(1, 31):
                for hour in range(0, 24):
                    s = year + '/' + str(month).zfill(2) + '/' + str(day).zfill(2) + " " + str(hour).zfill(2) + ":00:00"  ## zfill 指定字串長度
                    yearData.append(s)
    return yearData

for year in yearList:
    dateList = cdateList(year)
        
len(dateList)

4392

In [4]:
url = "https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0C480&stname=%25E6%25A1%2583%25E5%259C%2592&datepicker=2021-11-01"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, features="html.parser")

titles = soup.find_all("th")
titles = titles[11:28]
strtitle=[]
for title in titles:
    title = title.contents  ## 利用 contents 來定位標籤內的文字
    title=title[0] #+title[2]+title[4]
    strtitle.append(title)

In [5]:
def history_wether(station_name, station_num):
    new = []
    month31 = [1,3,5,7,8,10,12]
    for j in range(6, 12):
        if j in month31:
            for i in range(1,32):
                url = "https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station={}&stname=%25E6%25A1%2583%25E5%259C%2592&datepicker=2021-{}-{}".format(station_num, str(j).zfill(2), str(i).zfill(2))
                resp = requests.get(url)
                soup = BeautifulSoup(resp.text, features="html.parser")

                form = []
                soup = soup.tbody
                tmps = soup.find_all("tr")
                tmps = tmps[2:]
                for tmp in tmps:
                    tmp = tmp.find_all("td")
                    parameter =[]
                    for strtmp in tmp:
                        strtmp = strtmp.string
                        parameter.append(strtmp)
                    form.append(parameter)
                new.extend(form[1:25])  

        else:
            for i in range(1, 31):
                url = "https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station={}&stname=%25E6%25A1%2583%25E5%259C%2592&datepicker=2021-{}-{}".format(station_num, str(j).zfill(2), str(i).zfill(2))
                resp = requests.get(url)
                soup = BeautifulSoup(resp.text, features="html.parser")

                form = []
                soup = soup.tbody
                tmps = soup.find_all("tr")
                tmps = tmps[2:]
                for tmp in tmps:
                    tmp = tmp.find_all("td")
                    parameter =[]
                    for strtmp in tmp:
                        strtmp = strtmp.string
                        parameter.append(strtmp)
                    form.append(parameter)
                new.extend(form[1:25])  


#     new = pd.DataFrame(new, columns=strtitle)
#     data = new[["氣溫","相對溼度"]].replace("/\xa0", "None")
#     data["DataCreationDate"] = dateList
#     data = data.rename({"氣溫": "Temp", "相對溼度": 'Humidity'}, axis=1)
    new = pd.DataFrame(new, columns=strtitle)
    data = new[["氣溫","相對溼度"]].replace("/\xa0", "NULL")
    data = data.rename({"氣溫": "Temp", "相對溼度": "Humidity"}, axis=1)
    data["DataCreationDate"] = dateList
    data["Temp"] = data.Temp.str.split("\xa0", expand = True)[0]
    data["Humidity"] = data.Humidity.str.split("\xa0", expand = True)[0]
    data
    data.to_csv("C:/Users/user/OneDrive/桌面/Air/data/weather/history_{}_weather.csv".format(station_name), encoding = "utf_8_sig", index = False)
    # data

## 桃園 (C0C480)

In [6]:
Taoyuan = "C0C480"
history_wether("Taoyuan", "C0C480")

## 大園 (C0C540)

In [7]:
Dayuan = "C0C540"
history_wether("Dayuan", "C0C540")

## 觀音 (C0C590)

In [8]:
Guanyin = "C0C590"
history_wether("Guanyin", "C0C590")

## 平鎮 (C0C650)

In [9]:
Pingzhen = "C0C650"
history_wether("Pingzhen", "C0C650")

## 龍潭 (C0C670)

In [10]:
Longtan = "C0C670"
history_wether("Longtan", "C0C670")

## 中壢 (C0C700)

In [11]:
Zhongli = "C0C700"
history_wether("Zhongli", "C0C700")